In [2]:
import urllib
import re
import requests
from urllib.request import Request, urlopen
import numpy as np
import json
import datetime
from dateutil.parser import parse
import pprint
pp = pprint.PrettyPrinter(indent=4)
popUpList = []
information = ["vendor-name", 'type', 'price', 'attribute', 'about', 'menu', 'schedule', 'chef-info', 'stars-count', 'review-count', 'cuisine', 'cuisine-map', 'schedule']
eventVars = ['location', 'address', 'city', 'state', 'latitude', 'longitude', 'times']
timeVars = ['month', 'day', 'startTime', 'endTime']

In [3]:
def price_band(int_):
    if int_ < 25.00:
        return 1
    elif int_ < 35.00:
        return 2
    elif int_ < 60.00:
        return 3
    else:
        return 4

price_band(45)

3

In [4]:
url = 'https://eatfeastly.com/api/v1/meals/upcoming/?non_private=true&market=san-francisco'
req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
r = urlopen(req).read()
j = json.loads(r)['results']
for popUp in j:
    infoObj = dict.fromkeys(information)
    timeObj = dict.fromkeys(timeVars)
    timeObj['month'] = []
    timeObj['day'] = []
    timeObj['startTime'] = []
    timeObj['endTime'] = []
    eventObj = dict.fromkeys(eventVars)
    eventObj['times'] = timeObj
    infoObj['schedule'] = []
    
    popUpID = str(popUp['pk'])
    popUpUrl = 'https://eatfeastly.com/api/v1/meals/' + popUpID + '/?view=all'
    req = Request(popUpUrl, headers={'User-Agent': 'Mozilla/5.0'})
    r = urlopen(req).read()
    j = json.loads(r)
    
    infoObj['about'] = j['description']
    infoObj['vendor-name'] = j['title']
    infoObj['chef-info'] = j['cook']['about']
    cuisine = []
    for type in j['cuisine_type']:
        cuisine.append(type)
    infoObj['cuisine'] = cuisine
    menuStr = ''
    for dish in j['dishes']:
        menuStr = menuStr + dish['about'] + ' ' + dish['course'] + ' ' + dish['dish_title'] + ' '
    infoObj['menu'] = menuStr
    infoObj['price'] = price_band(float(j['price'][0]['price_feastly']))
    infoObj['review-count'] = j['cook']['review_count']
    eventObj['address'] = j['neighborhood']
    eventObj['city'] = j['location']['market']
    eventObj['latitude'] = j['location']['lat']
    eventObj['longitude'] = j['location']['lng']
    eventObj['state'] = j['location']['state']
    eventObj['times']['day'].append(parse(j['event_data']['start_time']).day)
    eventObj['times']['month'].append(parse(j['event_data']['start_time']).month)
    eventObj['times']['startTime'].append(parse(j['event_data']['start_time']).hour)
    duration = datetime.timedelta(minutes = j['event_data']['duration'])
    endTime = parse(j['event_data']['start_time']) + duration
    eventObj['times']['endTime'].append(endTime.hour)
    infoObj['schedule'].append(eventObj)
    infoObj['stars-count'] = j['cook']['review_avg']
    infoObj['type'] = 'popup'
    infoObj['attribute'] = {}
    infoObj['cuisine']
    
    popUpList.append(infoObj)

popUpList

[{'about': 'Pintxos are typically found in northern Spain. Every bar you walk into will have them lined up on their bar top and you can grab whichever you want yourself. Most are on perfectly toasted bread and toppings include tortilla espanola (Spanish omelette), chorizo, goat cheese with jam and many more. Paired perfectly with Cava, Spains sparkling wine. You can expect this with some California flare at our Pop-up!',
  'attribute': {},
  'chef-info': 'I have been in this industry for the past 6 years. I love everything about the hospitality field. I am especially passionate in the culinary department. In a recent trip to Barcelona I came back inspired to create a place where you feel like you are in Barcelona. That is where Champaneria Kharma gets its inspiration. Authentic and simple tapas paired with cava. I lived in Barcelona for three months this past year and learned so much about the food and culture and cannot wait to share with all of you!',
  'cuisine': ['spanish', 'califo

In [9]:
%store popUpList

UsageError: Unknown variable 'popup_list'


In [6]:
popUpJSON = json.dumps(popUpList)
with open('popup-data.txt', 'w') as outfile:
    json.dump(popUpJSON, outfile)

In [7]:

pp.pprint(popUpJSON)

('[{"vendor-name": "Pintxos de Espana", "type": "popup", "price": 3, '
 '"attribute": {}, "about": "Pintxos are typically found in northern Spain. '
 'Every bar you walk into will have them lined up on their bar top and you can '
 'grab whichever you want yourself. Most are on perfectly toasted bread and '
 'toppings include tortilla espanola (Spanish omelette), chorizo, goat cheese '
 'with jam and many more. Paired perfectly with Cava, Spains sparkling wine. '
 'You can expect this with some California flare at our Pop-up!", "menu": "The '
 'Champagne of Spain! Cava goes well with most everything and is great to pair '
 'with typical Spanish cuisine. Beverage Cava Roasted potatoes topped with '
 'aioli and chorizo Small Plate Pintxo - Papa con Chorizo Spanish Omelette '
 'with piquillo and a homemade mustard sauce Small Plate Pintxo - Tortilla con '
 'Piquillo Goat cheese with herbs with quince paste on top of toasted bread '
 'Small Plate Pintxo - Goat Cheese with quince Seared pork

 'quintessential clotted cream, and delightful sweets that are perfect to '
 'complement your selection of refreshing tea.", "menu": "Freshly baked scones '
 'served with homemade strawberry-rose preserve and 12-hour clotted cream '
 'Small Plate Fresh baked scones Salmon with lemon-chive cream cheese; smoky '
 'deviled eggs;  English tea cucumber Small Plate Assortment of savory '
 'sandwiches Tiramisu cake roll, freshly baked French madeleines; Viennese '
 'cream biscuits; strawberry rose cheesecake bites; mini chocolate tarts Small '
 'Plate Selection of delectable sweets Black, green, or fruit tea, served in '
 'traditional teaware Beverage Choice of teas ", "schedule": [{"location": '
 'null, "address": "South Beach", "city": "san-francisco", "state": '
 '"California", "latitude": 37.77493, "longitude": -122.41942, "times": '
 '{"month": [4], "day": [25], "startTime": [14], "endTime": [16]}}], '
 '"chef-info": "A non-chef by training and professional foodie by accident, '
 'June i

 'tofu, citrus Small Plate Black Cod Galangal  curry and sake kasu marinade, '
 'apple, arugula, roasted root vegetables Entree Lamb Chop Coconut milk, black '
 'sesame ice cream, ume Dessert Coconut Panna Cotta ", "schedule": '
 '[{"location": null, "address": "SOMA", "city": "san-francisco", "state": '
 '"California", "latitude": 37.77493, "longitude": -122.41942, "times": '
 '{"month": [4], "day": [25], "startTime": [19], "endTime": [21]}}], '
 '"chef-info": "With 20 years under his apron, Geoffrey Reed owes much of his '
 'culinary awareness to the ocean and mountains that surrounded him as a '
 'native of McKinleyville, California. It began early on in his parent\\u2019s '
 'garden, fishing in the creek behind the house and having full reign to play '
 'in the kitchen. Geoff got his first taste of the culinary life at 12, '
 'helping with chocolate dipped strawberries and washing dishes on '
 'Valentine\\u2019s Day at a restaurant where his older brother worked. That '
 'adventure

In [8]:
pp.pprint (json.loads(popUpJSON)[1])

{   'about': 'This menu is a celebration of local produce and sustainable '
             'fishermen that make living in California uniquely amazing. This '
             'menu blends tradition with modernity to showcase our Spring time '
             'bounty.',
    'attribute': {},
    'chef-info': 'Cooking, for me, started as a serious hobby and a fantastic '
                 'stress reliever from the rigors of graduate school. While '
                 'writing my PhD thesis, I realized that cooking was more than '
                 'a hobby but a true passion. I thus embarked on training in '
                 'some of the best restaurants in San Francisco. My years on '
                 'the line in these kitchens were truly some of that happiest '
                 'of my life.\r\n'
                 '\r\n'
                 'My culinary philosophy is directly connected to my training '
                 'as both a research scientist and cook. I only use locally '
                 'source